## 1. PropertyCreator (0.2 балла)
Напишите мета класс для создания свойств (property) класса из функций начинающихся с "set_", "get_" или "del_".

In [20]:
class Property(object):
    def __init__(self):
        self.fget = None
        self.fset = None
        self.fdel = None

In [40]:
class PropertyCreator(type): 

    def __new__(cls, name, bases, dct):
        # добавим все атрибуты, которые не нужно делать свойставми
        attrs = dict((name, value) for name, value in dct.items() if not (name.startswith(('set_', 'get_', 'del_'))))
                
        # найдем имена всех свойств
        properties = dict()
        for (name, value) in dct.items():
            if name.startswith(('set_', 'get_', 'del_')):
                properties[name[4:]] = Property()
               
        # сохраним для каждого свойства соответствующие функции
        for (name, value) in dct.items():
            if name.startswith('set_'):
                properties[name[4:]].fset = value
            elif name.startswith('get_'):
                properties[name[4:]].fget = value
            elif name.startswith('del_'):
                properties[name[4:]].fdel = value
                
        # добавим свойства с нужными функциями как атрибуты нового класса
        for (name, value) in properties.items():
            attrs[name] = property(fset=value.fset, fget=value.fget, fdel=value.fdel)
                    
        return type.__new__(cls, name, bases, attrs)

**Пример использования:**

In [44]:
class TestPropertyCreator(metaclass=PropertyCreator):
    def __init__(self, lo):
        self.__x = None
        self.lo = lo

    def get_x(self):
        return self.__x

    def set_x(self, value):
        if value < self.lo:
            raise ValueError("Value must in condition: {} <= value".format(self.lo))
        self.__x = value

    def del_x(self):
        self.__x = "No more"

In [45]:
obj = TestPropertyCreator(5) 
print(obj.lo)
obj.x = 6
print(obj.x)
del(obj.x)

5
6


**Протестируем:**

In [56]:
def test_simple():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self, lo):
            self.__x = None
            self.lo = lo

        def get_x(self):
            return self.__x

        def set_x(self, value):
            if value < self.lo:
                raise ValueError("Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"
    
    obj = TestPropertyCreator(5) 
    # проверим, что значение x не может быть ниже lo
    try:
        obj.x = 4 
        raise RuntimeError
    except ValueError:
        pass
    
    # проверим, что корректно присваивание и удаление
    obj.x = 6
    assert obj.x == 6
    del(obj.x)
    assert obj.x == 'No more'

In [57]:
test_simple()

In [98]:
def test_with_inheritance():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_x(self, value):
            self._secret_list.append("set")

    class TestPropertyCreatorInheritance(TestPropertyCreator, metaclass=PropertyCreator):
        pass
    
    obj = TestPropertyCreatorInheritance()
    obj.x = 5
    assert obj.x == 0
    assert obj._secret_list == ['set', 'get']
        

In [99]:
test_with_inheritance()

In [95]:
def test_partially_defined():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_y(self, value):
            self._secret_list.append("set")
            self._y = value
    
    
    obj = TestPropertyCreator()
    assert obj.x == 0
    obj.y = 1
    
    assert obj._secret_list == ['get', 'set']

In [96]:
test_partially_defined()

In [86]:
def test_sanity():
    class TestPropertyCreator(metaclass=PropertyCreator):
        _text = 0
        _boo = 'Boooooo'
        
        def get_raw_text(self):
             return self._boo

        def get_text(self):
             return self._text % 2

        def set_text(self, value):
            try:
                self._text = int(value)
            except ValueError:
                raise TypeError("unproper value for text: {}".format(value))
    
    obj = TestPropertyCreator()
    # проверим, что сеттер и геттер атрибута 'text' работают правильно
    try:
        obj.text = 'abc'
        raise RuntimeError
    except TypeError:
        pass
    obj.text = '10000'
    assert obj.text == 0
    
    # проверим, что у атрибута 'raw_text' работает только геттер
    assert obj.raw_text == 'Boooooo'
    try:
        obj.raw_text = '5'
        raise RuntimeError
    except AttributeError:
        pass

In [87]:
test_sanity()

In [90]:
def test_multiple_usages():
    class TestPropertyCreatorA(metaclass=PropertyCreator):
        def get_x(self):
            return 0
    class TestPropertyCreatorB(metaclass=PropertyCreator):
        def get_x(self):
            return 1
    class TestPropertyCreatorC(metaclass=PropertyCreator):
        def set_x(self, value):
            self.value = value + 1
        def get_x(self):
            return self.value
    
    
    obj1 = TestPropertyCreatorA()
    obj2 = TestPropertyCreatorB()
    obj3 = TestPropertyCreatorC()
    
    # проверим, что геттеры у 1 и 2 классов работают правильно
    assert obj1.x == 0
    assert obj2.x == 1
    
    # проверим, что сеттеры у 1 и 2 классов не установлены
    try:
        obj1.x = 1
        raise RuntimeError
    except AttributeError:
        pass
    try:
        obj2.x = 1
        raise RuntimeError
    except AttributeError:
        pass
    
    # проверим, что геттер и сеттер у 3 класса работают правильно
    obj3.x = 3
    assert obj3.x == 4
        

In [91]:
test_multiple_usages()

## 2. InstanceCountExeptioner (0.2 балла)
Напишите метакласс InstanceCountExeptioner, который будет следить за количеством экземпляров класса, использующих его. Количество задается через поле класса __max_instane_count__. Т. е. число экземпляров каждого класса регулируется отдельно. Если в классе не указано поле __max_instane_count__, то используйте заранее заданное число в метаклассе (любое).

In [1]:
class InstanceCountException(Exception):
    def __init__(self,*args,**kwargs):
        Exception.__init__(self,*args,**kwargs)

In [2]:
import functools

class InstanceCountExceptioner(type): 
    max_instance_count_default = 2
    
    def init_decorator(init_func):
        @functools.wraps(init_func)
        def init_wrapper(instance, *args, **kwargs):
            if instance.__class__.__max_instance_count__ <= 0:
                raise InstanceCountException
            instance.__class__.__max_instance_count__ -= 1
            init_func(instance, *args, **kwargs)
            
        return init_wrapper

    def __new__(cls, name, bases, dct):
        # добавим в список полей класса счетчик, если его там нет
        if '__max_instance_count__' not in dct.keys():
            dct['__max_instance_count__'] = InstanceCountExceptioner.max_instance_count_default
            
        # обернем __init__ с помощью декоратора, чтобы увеличивать счетчик при инициализации нового экземпляра класса
        dct['__init__'] = InstanceCountExceptioner.init_decorator(dct['__init__'])
                    
        return type.__new__(cls, name, bases, dct)

**Пример работы:**

In [3]:
class TestA(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 2
    def __init__(self, a):
        self.a = a
        
class TestB(metaclass=InstanceCountExceptioner): 
    __max_instance_count__ = 1
    def __init__(self, a): 
        self.a = a

a_one = TestA('one') 
a_two = TestA('two') 
b_one = TestB('one') # пока всё шло хорошо

# а вот 
try:
    a_three = TestA('three')  # выкенет исключение InstanceCountExeption
    raise RuntimeError
except InstanceCountException:
    pass

**Протестируем:**

In [4]:
class TestInstanceCountExceptionerA(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 3

    def __init__(self):
        self.a = 1

    def get(self):
        return self.a

class TestInstanceCountExceptionerB(metaclass=InstanceCountExceptioner):
    __max_instance_count__ = 2

    def __init__(self):
        self.b = 2

    def get(self):
        return self.b

In [5]:
def test_simple():
    a = TestInstanceCountExceptionerA() 
    assert a.get() == 1

def test_create():
    b = TestInstanceCountExceptionerB()
    assert b.get() == 2

def test_fail_create_a():
    a = TestInstanceCountExceptionerA()
    a = TestInstanceCountExceptionerA()
    try:
        a = TestInstanceCountExceptionerA()
        raise RuntimeError
    except InstanceCountException as e:
        pass


def test_fail_create_b():
    b = TestInstanceCountExceptionerB()
    try:
        b = TestInstanceCountExceptionerB()
        raise RuntimeError
    except InstanceCountException as e:
        pass

In [6]:
test_simple()
test_create()
test_fail_create_a()
test_fail_create_b()

## 3. JSONClassCreator (0.6 баллов)
Напишите метакласс, который будет по json представлению строить новый класс и обратно. Класс должен уметь следующее:

* Поддерживать сохранение и получение магических функций класса.
* Поддерживать сохранение и получение обычных функций.
* Поддерживать сохранение полей со стандартными типами.
* Уберите из сохранения следующие поля и методы: ['__dict__', '__weakref__', '__module__', '__init__']
* У создаваемого класса должна быть функция to_json_str

Формат json строки должен быть следующий:  
{  
"name": название класса,   
    "bases": базовые классы,   
    "methods": методы класса,  
    "attrs": поля класса  
}

Рекомендации:

* Для получения кода функций используйте модуль inspect.
* Для того, чтобы запустить код функций, можно использовать exec.
* Можно не исправлять ошибку типа OSError: could not get source code - возникает для функций, полученных с помощью exec.

In [189]:
import inspect
import json

class JSONClassCreator(type):   
    def __new__(mcls, json_str):
        json_obj = json.loads(json_str)
        name = json_obj['name']
        
        bases = tuple(globals()[base_name] for base_name in json_obj['bases'])
        
        attrs = dict()
        for key, value in json_obj['attrs'].items():
            attrs[key] = value
            
        attrs['to_json_str'] = JSONClassCreator.to_json_str
            
        for func_name, func_text in json_obj['methods'].items():
            exec(func_text) # теперь в локальном модуле существует нужная функция
            attrs[func_name] = locals()[func_name] # кладем ее в атрибуты
        
        return type.__new__(mcls, name, bases, attrs)

    def to_json_str(cls):
        exclude = ['__dict__', '__weakref__', '__module__', '__init__']       

        name = cls.__name__
        bases = list(base.__name__ for base in inspect.getmro(cls)[1:-1])
        
        methods, attrs = dict(), dict()
        for attr_key, attr_value in filter(lambda x: x[0] not in exclude, cls.__dict__.items()):
            if callable(attr_value):
                methods[attr_key] = inspect.getsource(attr_value).strip()
            else:
                attrs[attr_key] = attr_value
        
        return json.dumps({
            "name": name,
            "bases": bases,
            "methods": methods,
            "attrs": attrs
        })

    pass

**Пример генерации json по классу:**

In [190]:
class A:
    pass

class B(A):
    '''test B class'''
    y = 7
    
    def f(self):
        return 5

In [191]:
print(*json.loads(JSONClassCreator.to_json_str(B)).items(), sep="\n")

('name', 'B')
('bases', ['A'])
('methods', {'f': 'def f(self):\n        return 5'})
('attrs', {'__doc__': 'test B class', 'y': 7})


**Пример генерации класса по json:**

In [192]:
json_string = json.dumps({
    'name': 'Test',
    'bases': ['B'],
    'methods': {'f': 'def f(self, a):\n        return a + 5'},
    'attrs': {'x': 5}
})

In [193]:
tmp = JSONClassCreator(json_string)
obj = tmp()
print(obj.f(5), obj.x, obj.y)

10 5 7


**Протестируем на примере из условий:**

In [194]:
class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [195]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'f': 'def f(self, x):\n        print(x)', '__repr__': 'def __repr__(self):\n        return "Test(val={})".format(self.val)', '__str__': 'def __str__(self):\n        return "Test(val={})".format(self.val)'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})


In [196]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

hi


(Test(val=[1, 2, 3]), None, [1, 2, 3], 'Тестовый класс')

In [197]:
tmp.__dict__

mappingproxy({'__doc__': 'Тестовый класс',
              'val': [1, 2, 3],
              'to_json_str': <function __main__.JSONClassCreator.to_json_str(cls)>,
              'f': <function __main__.f(self, x)>,
              '__repr__': <function __main__.__repr__(self)>,
              '__str__': <function __main__.__str__(self)>,
              '__module__': '__main__'})